<a href="https://colab.research.google.com/github/lucaCosca/JetNet_ML-Final-Project/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>